### Import packages - libraries

In [1]:
import pandas as pd
import numpy as np
import time
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from datasketch import MinHashLSH
from sklearn.feature_extraction.text import CountVectorizer
import numpy.linalg as LA
#import spacy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from datasketch import MinHash
import nltk
#nltk.download('stopwords')
#nlp = spacy.load('en_core_web_md')

In [2]:
train_data = pd.read_csv('datasets/q2a/corpusTrain.csv', sep=',')
test_data = pd.read_csv('datasets/q2a/corpusTest.csv', sep=',')
train_data = train_data
test_data = test_data

In [3]:
len(train_data)

531990

In [4]:
len(test_data)

5374

### Preprocess

In [3]:
def preprocess(text):
    stopWords = stopwords.words('english')
    p_stemmer = SnowballStemmer(language='english')
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    tokens = [wrd for wrd in tokens if(wrd not in stopWords)]
    stems = [p_stemmer.stem(t) for t in tokens]
    return stems

In [4]:
def preprocess2(text):
    stopWords = stopwords.words('english')
    p_stemmer = SnowballStemmer(language='english')
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    tokens = [wrd for wrd in tokens if(wrd not in stopWords)]
    stems = [p_stemmer.stem(t) for t in tokens]
    stems = " ".join(stems)
    return stems

### Exact Cosine 

In [5]:
vectorizer = TfidfVectorizer(tokenizer=preprocess2)
flag = 0

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


def exact_cosine(train_data,test_data):
    counter = 0
    t0 = time.time()
    line = 0
    for d in test_data['Content']:
        for d2 in train_data['Content']:
            c = cosine_sim(d,d2)
            if(c > 0.8):
                counter = counter + 1;
                break;
    t1 = time.time()
                
    print(counter)
    print(t1-t0)
    
cos = lambda a, b : round(np.inner(a, b)/(LA.norm(a)*LA.norm(b)), 3)

vectorizer = CountVectorizer(tokenizer = preprocess)

def exact_cosine2(trainVectorizerArray,testVectorizerArray):
    counter2 = 0
    #train = train_data['Content']
    #test = test_data['Content']
    #trainVectorizerArray = vectorizer.fit_transform(train).toarray()
    for testV in testVectorizerArray:
        for vector in trainVectorizerArray:
            cosine = cos(vector, testV)
            if cosine > 0.8:
                counter2 = counter2 + 1
                continue
    return counter2

In [6]:
test = test_data['Content'].head(1000)
train = train_data['Content'].head(50000)
trainVectorizerArray = vectorizer.fit_transform(train).toarray()
testVectorizerArray = vectorizer.transform(test).toarray()

In [7]:
counter = 0

t0 = time.time()

counter = exact_cosine2(trainVectorizerArray,testVectorizerArray)

t1 = time.time()
total = t1-t0



print("exact cosine:", end=' ')
print(counter)
print("time: ",end=' ')
print(total)

c:\python\python37\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


exact cosine: 336
time:  24418.88220191002


### Exact Jaccard

In [5]:
def jaccard_similarity(x, y):
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)

In [6]:
test = test_data['Content'].head(1000)
train = train_data['Content'].head(50000)

In [12]:
trainD = train.map(preprocess2)
testD = test.map(preprocess2)

In [17]:
counterJ = 0
t3 = time.time()
for testS in testD:
    for trainS in trainD:
        j_sim = jaccard_similarity(testS,trainS)
        j_sim = round(j_sim,2)
        if j_sim>0.8:
            counterJ = counterJ+1
            break
t4 = time.time()
tot_time = t4-t3

print("exact jaccard:", end=' ')
print(counterJ)
print("time: ",end=' ')
print(tot_time)

exact jaccard: 931
time:  30.483662366867065
